In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('./training_solutions_rev2.csv')

df_train, df_test = train_test_split(df, test_size=.2)
df_train.shape, df_test.shape

((546, 38), (137, 38))

In [8]:
from skimage.transform import resize
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
          

ORIG_SHAPE = (424,424)
CROP_SIZE = (256,256)
IMG_SHAPE = (64,64)

def get_image(path, x1,y1, shape, crop_size):
    x = plt.imread(path)
    x = x[x1:x1+crop_size[0], y1:y1+crop_size[1]]
    x = resize(x, shape)
    x = x/255.
    return x

def get_all_images(dataframe, shape=IMG_SHAPE, crop_size=CROP_SIZE):
    x1 = (ORIG_SHAPE[0]-CROP_SIZE[0])//2
    y1 = (ORIG_SHAPE[1]-CROP_SIZE[1])//2
   
    sel = dataframe.values
    ids = sel[:,0].astype(int).astype(str)
    y_batch = sel[:,1:]
    x_batch = []
    for i in tqdm(ids):
        x = get_image('./images_training_rev2/'+i+'.jpg', x1,y1, shape=shape, crop_size=crop_size)
        x_batch.append(x)
    x_batch = np.array(x_batch)
    return x_batch, y_batch

X_train, y_train = get_all_images(df_train)
X_test, y_test = get_all_images(df_test)

100%|██████████| 137/137 [00:01<00:00, 116.31it/s]


In [10]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, GlobalMaxPooling2D
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

model = Sequential()
model.add(Conv2D(512, (3, 3), input_shape=(IMG_SHAPE[0], IMG_SHAPE[1], 3)))
model.add(Conv2D(256, (3, 3)))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Conv2D(128, (3, 3)))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Conv2D(128, (3, 3)))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(GlobalMaxPooling2D())


model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(37))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=[root_mean_squared_error])
model.summary()

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 512)       14336     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 256)       1179904   
_________________________________________________________________
activation_1 (Activation)    (None, 60, 60, 256)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 256)       590080    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 26, 26, 128)       295040    
_________________________________________________________________
activation_2 (Activation)    (None, 26, 26, 128)      

In [11]:
batch_size = 128
model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

Train on 546 samples, validate on 137 samples
Epoch 1/30
546/546 [==============================] - ETA: 3:40 - loss: 0.6931 - root_mean_squared_error: 0.435 - ETA: 2:41 - loss: 0.6914 - root_mean_squared_error: 0.432 - ETA: 2:21 - loss: 0.6741 - root_mean_squared_error: 0.423 - ETA: 2:05 - loss: 0.6610 - root_mean_squared_error: 0.413 - ETA: 1:52 - loss: 0.6466 - root_mean_squared_error: 0.404 - ETA: 1:41 - loss: 0.6348 - root_mean_squared_error: 0.397 - ETA: 1:30 - loss: 0.6198 - root_mean_squared_error: 0.389 - ETA: 1:21 - loss: 0.6103 - root_mean_squared_error: 0.380 - ETA: 1:11 - loss: 0.5974 - root_mean_squared_error: 0.372 - ETA: 1:02 - loss: 0.5847 - root_mean_squared_error: 0.365 - ETA: 53s - loss: 0.5731 - root_mean_squared_error: 0.359 - ETA: 44s - loss: 0.5625 - root_mean_squared_error: 0.35 - ETA: 35s - loss: 0.5528 - root_mean_squared_error: 0.34 - ETA: 26s - loss: 0.5425 - root_mean_squared_error: 0.33 - ETA: 18s - loss: 0.5333 - root_mean_squared_error: 0.33 - ETA: 9s -

KeyboardInterrupt: 